In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
credentials_location = '/root/.google/credentials/dezoomcamp-ikg-764b429f1606.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/mnt/d/apache/spark-3.4.2-bin-hadoop3/jars/gcs-connector-hadoop3-latest.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/03/02 15:56:33 WARN Utils: Your hostname, Desktop-Gar resolves to a loopback address: 127.0.1.1; using 172.25.243.204 instead (on interface eth0)
24/03/02 15:56:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/02 15:56:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 15:56:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
df_green = spark.read.csv('gs://nytaxi-dezoomcamp/green_tripdata_2020-01.csv.gz', header=True) 


In [6]:
df_green.count()

447770

In [7]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-12-18 15:52:30|  2019-12-18 15:54:39|                 N|         1|         264|         264|              5|          .00|        3.5|  0.5|    0.

In [8]:
df_green.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



In [9]:
df = spark.read.csv('gs://nytaxi-dezoomcamp/green_tripdata_2020-01.csv.gz', header=True) 


In [10]:
df = df.repartition(16)

In [11]:
df.write.parquet('gs://nytaxi-dezoomcamp/pq/green/2020/01/', mode='overwrite')

24/03/02 15:59:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 15:59:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [12]:
df = spark.read \
    .option("header", "true") \
    .csv('gs://nytaxi-dezoomcamp/yellow_tripdata_2020-01.csv.gz')

In [13]:
df = df.repartition(50)

In [14]:
df.write.parquet('gs://nytaxi-dezoomcamp/pq/yellow/2020/01/', mode='overwrite')

24/03/02 16:02:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 16:02:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 16:02:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 16:03:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 16:03:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 16:03:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 16:03:11 WARN MemoryManager: Total allocation exceeds 95.00% 

In [15]:
!gsutil ls -lh gs://nytaxi-dezoomcamp/pq/green/2020/01/



Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update

       0 B  2024-03-02T08:59:41Z  gs://nytaxi-dezoomcamp/pq/green/2020/01/
       0 B  2024-03-02T08:59:41Z  gs://nytaxi-dezoomcamp/pq/green/2020/01/_SUCCESS
782.37 KiB  2024-03-02T08:59:35Z  gs://nytaxi-dezoomcamp/pq/green/2020/01/part-00000-5072de0d-765e-49bd-afbb-3c8abf3c21c7-c000.snappy.parquet
 784.9 KiB  2024-03-02T08:59:35Z  gs://nytaxi-dezoomcamp/pq/green/2020/01/part-00001-5072de0d-765e-49bd-afbb-3c8abf3c21c7-c000.snappy.parquet
785.52 KiB  2024-03-02T08:59:36Z  gs://nytaxi-dezoomcamp/pq/green/2020/01/part-00002-5072de0d-765e-49bd-afbb-3c8abf3c21c7-c000.snappy.parquet
784.99 KiB  2024-03-02T08:59:36Z  gs://nytaxi-dezoomcamp/pq/green/2020/01/part-00003-5072de0d-765e-49bd-afbb-3c8abf3c21c7-c000.snappy.parquet
 783.9 KiB  2024-03-02T08:59:36Z  gs://nytaxi-dezoomcamp/pq/green/2020/01/part-00004-5072de0d-765e-49bd-afbb-3c8abf3c21c7-c000.snappy.parquet
782.79 KiB

In [16]:
!gsutil ls -lh gs://nytaxi-dezoomcamp/pq/yellow/2020/01/

       0 B  2024-03-02T09:04:09Z  gs://nytaxi-dezoomcamp/pq/yellow/2020/01/
       0 B  2024-03-02T09:04:09Z  gs://nytaxi-dezoomcamp/pq/yellow/2020/01/_SUCCESS
   3.3 MiB  2024-03-02T09:03:53Z  gs://nytaxi-dezoomcamp/pq/yellow/2020/01/part-00000-4d90723b-db59-4a7c-a903-c9271b926eb3-c000.snappy.parquet
  3.31 MiB  2024-03-02T09:03:53Z  gs://nytaxi-dezoomcamp/pq/yellow/2020/01/part-00001-4d90723b-db59-4a7c-a903-c9271b926eb3-c000.snappy.parquet
  3.32 MiB  2024-03-02T09:03:54Z  gs://nytaxi-dezoomcamp/pq/yellow/2020/01/part-00002-4d90723b-db59-4a7c-a903-c9271b926eb3-c000.snappy.parquet
  3.31 MiB  2024-03-02T09:03:54Z  gs://nytaxi-dezoomcamp/pq/yellow/2020/01/part-00003-4d90723b-db59-4a7c-a903-c9271b926eb3-c000.snappy.parquet
   3.3 MiB  2024-03-02T09:03:54Z  gs://nytaxi-dezoomcamp/pq/yellow/2020/01/part-00004-4d90723b-db59-4a7c-a903-c9271b926eb3-c000.snappy.parquet
  3.31 MiB  2024-03-02T09:03:54Z  gs://nytaxi-dezoomcamp/pq/yellow/2020/01/part-00005-4d90723b-db59-4a7c-a903-c9271b926eb3-c0